In [12]:
pip install pennylane

Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np
import torch
import torch.nn as nn
import pennylane as qml

In [14]:
# XOR dataset
XOR_INPUT = np.array([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=np.float32)
XOR_TARGET = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [15]:
# Convert the input and target data to PyTorch tensors
X = torch.from_numpy(XOR_INPUT).view(1, 4, 2)  # Add a batch and sequence dimension
Y = torch.from_numpy(XOR_TARGET).view(1, 4, 1)  # Make sure it has the correct shape

In [16]:
class QRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_qubits=4, n_qlayers=1, batch_first=True, return_sequences=False,
                 return_state=False, backend="default.qubit"):
        super(QRNN, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.backend = backend

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state

        self.wires = [f"wire_{i}" for i in range(self.n_qubits)]
        self.dev = qml.device(self.backend, wires=self.wires)

        def _circuit(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires]

        self.qlayer = qml.QNode(_circuit, self.dev, interface="torch")

        weight_shapes = {"weights": (n_qlayers, n_qubits)}

        self.clayer_in = torch.nn.Linear(self.concat_size, n_qubits)
        self.VQC = qml.qnn.TorchLayer(self.qlayer, weight_shapes)
        self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)

    def forward(self, x, init_states=None):
        if self.batch_first:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size)  # hidden state (output)
        else:
            h_t = init_states[0][0]

        for t in range(seq_length):
            x_t = x[:, t, :]
            v_t = torch.cat((h_t, x_t), dim=1)
            y_t = self.clayer_in(v_t)

            h_t = torch.tanh(self.clayer_out(self.VQC(y_t)))

            hidden_seq.append(h_t.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()

        return hidden_seq, (h_t,)

In [17]:
# Instantiate the QRNN model
input_size = 2
hidden_size = 4
qrnn_model = QRNN(input_size=input_size, hidden_size=hidden_size)

In [18]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(qrnn_model.parameters(), lr=0.01)

In [19]:
# Training loop
epochs = 200
for epoch in range(epochs):
    # Forward pass
    outputs, _ = qrnn_model(X)

    # Compute the loss
    loss = criterion(outputs, Y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: 0.8013659715652466
Epoch 100, Loss: 0.014955466613173485


In [20]:
# Testing the QRNN on XOR data
with torch.no_grad():
    test_outputs, _ = qrnn_model(X)
    for i in range(len(XOR_INPUT)):
        input_data = XOR_INPUT[i]
        output = test_outputs[0, i, 0].item()
        target = XOR_TARGET[i, 0]
        print(f'Input: {input_data}, Output: {output}, Target: {target}')


Input: [0. 0.], Output: 0.012531504034996033, Target: 0.0
Input: [1. 0.], Output: 0.9301008582115173, Target: 1.0
Input: [0. 1.], Output: 0.9324067831039429, Target: 1.0
Input: [1. 1.], Output: 0.012725306674838066, Target: 0.0


Multi-Tasking

In [21]:
# Set print options to suppress scientific notation
np.set_printoptions(suppress=True)

# XOR, OR, AND dataset
INPUT_DATA = np.array([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=np.float32)
X = torch.from_numpy(INPUT_DATA).view(1, 4, 2)  # Add a batch and sequence dimension

# Combined targets for XOR, OR, AND
TARGET_XOR = np.array([[0], [1], [1], [0]], dtype=np.float32)
TARGET_OR = np.array([[0], [1], [1], [1]], dtype=np.float32)
TARGET_AND = np.array([[0], [0], [0], [1]], dtype=np.float32)

Y = torch.from_numpy(np.concatenate([TARGET_XOR, TARGET_OR, TARGET_AND], axis=1)).view(1, 4, 3)  # Three output dimensions

In [22]:
class QRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_qubits=4, n_qlayers=1, batch_first=True, return_sequences=False,
                 return_state=False, backend="default.qubit"):
        super(QRNN, self).__init__()
        self.n_inputs = input_size
        self.hidden_size = hidden_size
        self.concat_size = self.n_inputs + self.hidden_size
        self.n_qubits = n_qubits
        self.n_qlayers = n_qlayers
        self.backend = backend

        self.batch_first = batch_first
        self.return_sequences = return_sequences
        self.return_state = return_state

        self.wires = [f"wire_{i}" for i in range(self.n_qubits)]
        self.dev = qml.device(self.backend, wires=self.wires)

        def _circuit(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires]

        self.qlayer = qml.QNode(_circuit, self.dev, interface="torch")

        weight_shapes = {"weights": (n_qlayers, n_qubits)}

        self.clayer_in = torch.nn.Linear(self.concat_size, n_qubits)
        self.VQC = qml.qnn.TorchLayer(self.qlayer, weight_shapes)
        self.clayer_out = torch.nn.Linear(self.n_qubits, self.hidden_size)
        self.output_layer = torch.nn.Linear(self.hidden_size, 3)  # Three output dimensions

    def forward(self, x, init_states=None):
        if self.batch_first:
            batch_size, seq_length, features_size = x.size()
        else:
            seq_length, batch_size, features_size = x.size()

        hidden_seq = []
        if init_states is None:
            h_t = torch.zeros(batch_size, self.hidden_size)  # hidden state (output)
        else:
            h_t = init_states[0][0]

        for t in range(seq_length):
            x_t = x[:, t, :]
            v_t = torch.cat((h_t, x_t), dim=1)
            y_t = self.clayer_in(v_t)

            h_t = torch.tanh(self.clayer_out(self.VQC(y_t)))

            hidden_seq.append(h_t.unsqueeze(0))

        hidden_seq = torch.cat(hidden_seq, dim=0)
        hidden_seq = hidden_seq.transpose(0, 1).contiguous()

        # Apply the output layer
        outputs = self.output_layer(hidden_seq)

        return outputs, (h_t,)

In [24]:
# Instantiate the QRNN model
input_size = 2
hidden_size = 4
qrnn_model = QRNN(input_size=input_size, hidden_size=hidden_size)

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(qrnn_model.parameters(), lr=0.01)

In [25]:
# Training loop
epochs = 200
for epoch in range(epochs):
    # Forward pass
    outputs, _ = qrnn_model(X)

    # Compute the loss
    loss = criterion(outputs, Y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every 100 epochs
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 0.30112624168395996
Epoch 100, Loss: 6.616481550736353e-05


In [26]:
# Testing the QRNN on XOR, OR, AND data
with torch.no_grad():
    test_outputs, _ = qrnn_model(X)
    for i in range(len(INPUT_DATA)):
        input_data = INPUT_DATA[i]
        output = test_outputs[0, i, :].numpy()
        target = Y[0, i, :].numpy()
        print(f'Input: {input_data}, Output: {output}, Target: {target}')


Input: [0. 0.], Output: [-0.0000069   0.00008124  0.00002685], Target: [0. 0. 0.]
Input: [1. 0.], Output: [ 0.99974895  1.0002161  -0.00022137], Target: [1. 1. 0.]
Input: [0. 1.], Output: [1.0002334  0.99973917 0.00020742], Target: [1. 1. 0.]
Input: [1. 1.], Output: [-0.000076    0.99986285  0.99988174], Target: [0. 1. 1.]
